   1  : 1.58,
   0.1: 2.71,
  0.05: 3.84, 
  0.01: 6.63,
 0.005: 7.88, 
 0.001: 10.83

In [35]:
# 2.71 - 1.13

In [1]:
import pandas as pd
import numpy as np
from joblib import dump

from chi import square as cs

In [2]:
data = pd.read_excel("komentar/data_clean_goejk.xlsx")
komentar = data.komentar.tolist()
label = np.array(data.label.tolist())
y = label
data.tail()

,Unnamed: 0,label,komentar,komentar_lama
3205,3205,0,yuk firalkan biar makin pacu buat baik tuju in...,"yuk firalkan, biar semakin memacu kita untuk b..."
3206,3206,1,yukk bantu bapaknyaa,Yukkk bantu bapaknyaa!
3207,3207,1,yukk semangat decacorn yayaya a,Yukkk semangat Decacorn @gojekindonesia @tokop...
3208,3208,1,yummeereh gin 40k aja thank you,yummeeereh segini 40k aja thank you @gojekindo...
3209,3209,1,yuuhuu mission completed nih moga masuk orang ...,yuuhuuuu..\nMission completed nih @gojekindone...


# Ekstraksi dan Seleksi Fitur

In [3]:
sf = cs.ChiSquare(alpha = 0.1)
sf.find_best_features(komentar, label, fitur = None)

In [4]:
print("jumlah fitur terbaik:",len(sf.best_features),"dari",len(sf.fitur), "fitur")
best_features = sf.best_features

jumlah fitur terbaik: 1492 dari 6351 fitur


# Pemodelan

## TF-IDF

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(vocabulary = best_features)#
X = vectorizer.fit_transform(komentar)
save = vectorizer.fit(komentar)
dump(save, "model/tfidf.w")
# print(vectorizer.get_feature_names())
fitur = vectorizer.get_feature_names()
print(X.shape)

(3210, 1492)


## Pemodelan MNB 

In [18]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

# Pemodelan dan Validasi Model

In [19]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

skf = StratifiedKFold(n_splits=10)
skf.get_n_splits(X, y)

start = 0.001
stop = 1
step = start
alpha_list = np.arange(start,stop+step,step)
# print(alpha_list)
akurasi = list()

ix = 0 #just counter
for alpha in alpha_list:
    clf.set_params(alpha=alpha)
    temp_akurasi = list()
    for train_index, test_index in skf.split(X, y):
    #     print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        #latih model
        clf.fit(X_train,y_train)
        
        #prediksi
        y_pred = clf.predict(X_test)
        temp_akurasi.append(accuracy_score(y_test, y_pred))
    rerata_akurasi = sum(temp_akurasi)/len(temp_akurasi)
    akurasi.append(rerata_akurasi)
#     print('alpha   :',alpha)
#     print('     akurasi :',rerata_akurasi)
#     print("="*100)
    ix +=1
    if ix%100 !=0:
        print(".", end="")
    else:
#         print(alpha, ' | ', rerata_akurasi)
        print("")

        pass

ditt = {
    "alpha":alpha_list,
    "akurasi":akurasi
}
index = akurasi.index(max(akurasi))
acc = pd.DataFrame.from_dict(ditt)
acc

...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................
...................................................................................................


,alpha,akurasi
0,0.001,0.831464
1,0.002,0.831776
2,0.003,0.832710
3,0.004,0.832710
4,0.005,0.833022
...,...,...
995,0.996,0.780374
996,0.997,0.780374
997,0.998,0.780374
998,0.999,0.780374


In [23]:
print(alpha_list[index])
print("Akurasi",akurasi[index])

0.042
Akurasi 0.8358255451713396


In [21]:
# print(alpha_list[index])
# print(akurasi[index])

# Menyimpan Model

In [22]:
clf.set_params(alpha=alpha_list[index])
clf.fit(X, y)
dump(clf, "model/mnb.m")

['model/mnb.m']